# Flower Classification with MobileNetV2

## Project Rules Checklist
- **Architecture**: MobileNetV2 (Transfer Learning)
- **Dataset**: tf_flowers
- **Preprocessing**: 224x224, Normalized (0-1)
- **Training**: >10 Epochs, Batch 32, Adam Optimizer
- **Features**: Grad-CAM, Fine-Tuning, TFLite Export
- **Split**: 80% Train (with Val), 20% Test

In [ ]:
# 1. Environment Setup & Drive Mount
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import shutil
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
    
    # Create Project Directory in Drive
    project_root = '/content/drive/MyDrive/flower-classification-app'
    model_dir = os.path.join(project_root, 'model')
    os.makedirs(model_dir, exist_ok=True)
    print(f"Project directory created/verified at: {project_root}")
    print(f"Models will be saved to: {model_dir}")
    print("Access your files at: https://drive.google.com/drive/my-drive")
    
    checkpoint_path = os.path.join(model_dir, 'flower_model_best.keras')
    final_model_path = os.path.join(model_dir, 'flower_model_final.keras')
    tflite_path = os.path.join(model_dir, 'flower_model.tflite')

except ImportError:
    print("Not running in Google Colab or Drive already mounted.")
    checkpoint_path = 'flower_model_best.keras'
    final_model_path = 'flower_model_final.keras'
    tflite_path = 'flower_model.tflite'

# Install split-folders for easy data splitting
!pip install split-folders
import splitfolders

print(f"TensorFlow Version: {tf.__version__}")

In [ ]:
# 2. Dataset Preparation (tf_flowers)
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_file = tf.keras.utils.get_file(origin=dataset_url, fname='flower_photos', untar=True)
print(f"Dataset downloaded to: {data_file}")

# Robustly find the directory containing the class folders
base_extract_dir = pathlib.Path(data_file).parent
data_dir = None

# Look for the folder that contains specific flower classes
for root, dirs, files in os.walk(base_extract_dir):
    if 'daisy' in dirs and 'dandelion' in dirs:
        data_dir = pathlib.Path(root)
        break

if data_dir is None:
    # Fallback attempt
    possible_dir = base_extract_dir / 'flower_photos'
    if possible_dir.exists():
        data_dir = possible_dir

if data_dir is None or not data_dir.exists():
    raise ValueError("Could not verify dataset structure. Check download path.")

print(f"Verified dataset source directory: {data_dir}")
print(f"Classes found: {[d.name for d in data_dir.iterdir() if d.is_dir()]}")

# Output directory for split data
output_dir = '/content/flower_data_split'

# FORCE CLEANUP: Remove existing split folder to ensure fresh start
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print(f"Removed existing {output_dir}")

# Split Data: 80% Train (Validation comes from this), 20% Test
# We will split into: Train (64%), Val (16%), Test (20%) -> Total 100%
print("Splitting dataset... (this may take a moment)")
splitfolders.ratio(data_dir, output=output_dir, seed=1337, ratio=(.64, .16, .2), group_prefix=None)
print("Split completed.")

train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

In [ ]:
# 3. Data Preprocessing & Augmentation
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Training Generator with Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Optimized for MobileNetV2        # Normalize 0-1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation & Test Generators (Rescale Only)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

print("Loading Training Data...")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("Loading Validation Data...")
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("Loading Test Data...")
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Important for evaluation
)

if train_generator.n == 0:
    raise ValueError(f"No training images found in {train_dir}. Please check the dataset path.")
else:
    print(f"Found {train_generator.n} training images.")

class_names = list(train_generator.class_indices.keys())
print("Classes:", class_names)

# 4. Model Architecture (Transfer Learning)
Using **MobileNetV2** pre-trained on ImageNet as the base. We freeze the base layers and add a custom classification head.

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model
base_model.trainable = False

# Custom Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# 5. Training (Stage 1)

# Use the path defined in Step 1
print(f"Saving best model to: {checkpoint_path}")

# Callbacks
checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

print("Starting Training...")
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stop]
)

In [ ]:
# Visualization
def plot_history(history, title="Model Performance"):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.suptitle(title)
    plt.show()

plot_history(history, "Initial Training")

# 6. Fine-Tuning (Stage 2)
Unfreezing the top layers of the base model to adapt them to the flower features.

In [ ]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

# Compile with Lower Learning Rate
model.compile(optimizer=Adam(learning_rate=1e-5),  # Low LR for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 10
total_epochs =  len(history.history['accuracy']) + fine_tune_epochs

history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=val_generator,
    callbacks=[checkpoint, early_stop]
)

plot_history(history_fine, "Fine-Tuning Performance")

# 7. Evaluation & Confusion Matrix

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Confusion Matrix
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.ylabel('Prediction')
plt.xlabel('Truth')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred, target_names=class_names))

# 8. Model Explainability (Grad-CAM)
Visualizing which parts of the image the model focuses on.

In [ ]:
import cv2

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = heatmap * alpha + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype('uint8')

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(superimposed_img)
    plt.title("Grad-CAM")
    plt.axis('off')
    plt.show()

# Get last conv layer from MobileNetV2
last_conv_layer_name = "out_relu" # This is typically the last conv block in MobileNetV2

# Pick a sample image from test set
img_path = os.path.join(test_dir, class_names[0], os.listdir(os.path.join(test_dir, class_names[0]))[0])
img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
display_gradcam(img_path, heatmap)

# 9. Model Export (TFLite)
Converting the model for mobile deployment.

In [ ]:
# Save Keras Model
print(f"Saving final model to {final_model_path}...")
model.save(final_model_path)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] # Dynamic range quantization
tflite_model = converter.convert()

with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model saved as {tflite_path} with size:", len(tflite_model), "bytes")